In [2]:
# ======================================
# 0. 기본 라이브러리
# ======================================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from bs4 import BeautifulSoup
import requests
import openpyxl
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import time
from selenium.webdriver.common.by  import By
from selenium.webdriver.common.keys import Keys

import matplotlib.font_manager as fm
from matplotlib import rcParams
from pathlib import Path

# ======================================
# 1. 스타일 먼저 (중요: 폰트보다 먼저)
# ======================================
plt.style.use('_mpl-gallery-nogrid')   # matplotlib 스타일
sns.set_theme(style="white")           # seaborn 기본 테마

# ======================================
# 2. 폰트 설정 (나눔고딕)
# ======================================
FONT_PATH = r"C:/Users/user/Desktop/seoul_pv_load_analysis/NanumGothic.ttf"

# matplotlib에 폰트 등록
fm.fontManager.addfont(FONT_PATH)

# 전역 폰트 지정
rcParams["font.family"] = "NanumGothic"

# 마이너스 깨짐 방지
rcParams["axes.unicode_minus"] = False

# (선택) 기본 폰트 사이즈 살짝 키우기
rcParams["font.size"] = 12
rcParams["axes.titlesize"] = 18
rcParams["axes.labelsize"] = 14
rcParams["xtick.labelsize"] = 12
rcParams["ytick.labelsize"] = 12

# ======================================
# 3. 개별 객체용 폰트 (pie, title 등)
# ======================================
font_prop = fm.FontProperties(fname=FONT_PATH)

# ======================================
# 4. seaborn스타일 설정
# ======================================
sns.set_style("ticks") # 스타일 테마 설정
sns.set_context("notebook") # 문맥에 따라 스타일 크기 조정
sns.set_palette("pastel")

In [ ]:
# DAY 1 목표:
# 1) CSV가 정상 로드되는지 확인하고
# 2) 시간축(datetime / hour / month)을 만들 수 있는지 검증하고
# 3) 분석 가능한 최소 테이블(df_base)을 만든다.

In [ ]:
# 1. 라이브러리 + 경로 기본 세팅

# 1) "현재 작업 위치"를 기준으로 프로젝트 루트를 잡는다.
    # 어느 장소든 경로가 달라도 코드가 잘 돌아가게 하기 위함
    # 노트북이 있는 폴더를 기준으로 한다.
NOTEBOOK_DIR = Path.cwd()

# 2) 프로젝트 루트 후보로 잡는다.
    # 보통 notebooks 폴더에서 실행하므로 부모 폴더가 프로젝트 루트일 가능성이 높다.
PROJECT_ROOT_CANDIDATE_1 = NOTEBOOK_DIR
PROJECT_ROOT_CANDIDATE_2 = NOTEBOOK_DIR.parent

# 3) data 폴더가 어디에 있는지 먼저 "탐색" 한다.
    # data 폴더가 NOTEBOOK_DIR 아래에 있거나, 부모 폴더 아래에 있을 수 있다.
DATA_DIR_1 = PROJECT_ROOT_CANDIDATE_1 / "data"
DATA_DIR_2 = PROJECT_ROOT_CANDIDATE_2 / "data"

# 4) 실제 존재하는 data 폴더를 선택한다.
if DATA_DIR_1.exists():
    DATA_DIR = DATA_DIR_1
elif DATA_DIR_2.exists():
    DATA_DIR = DATA_DIR_2
else:
    # data 폴더를 못 찾으면 여기서 중단하고, 폴더 구조를 먼저 확인해야 한다.
    raise FileNotFoundError(
        "data 폴더가 없습니다. \n"
        f"- 현재 위치: {NOTEBOOK_DIR}\n"
        f"- 확인한 경로1: {DATA_DIR_1}\n"
        f"- 확인한 경로2: {DATA_DIR_2}\n"
        "해결: 프로젝트 폴더 안에 data 폴더가 있는지 확인" )

# 5) 잘 잡혔는지 출력해서 확인한다. (중요)
print("NOTEBOOK_DIR = ", NOTEBOOK_DIR)
print("DATA_DIR = ", DATA_DIR)

NOTEBOOK_DIR =  c:\Users\user\Desktop\seoul_pv_load_analysis\notebooks
DATA_DIR =  c:\Users\user\Desktop\seoul_pv_load_analysis\data


In [ ]:
# 2. 데이터 파일 찾기 (파일명 모르면 자동 탐색)

# 1) data 폴더 안에 있는 파일 목록을 확인한다.
data_files = sorted(DATA_DIR.glob("*"))

# 2) 파일이 하나도 없으면 로드할 게 없으니 중단한다.
if len(data_files) == 0:
    raise FileNotFoundError(f"data 폴더({DATA_DIR}) 안에 파일이 없습니다. CSV를 넣었는지 확인")

# 3) 파일 목록을 출력 해보고,  우리가 쓸 CSV가 있는지 확인한다.
print("===data 폴더 파일 목록===")
for f in data_files:
    print("-", f.name)

===data 폴더 파일 목록===
- 법정동별시간별전력사용량.csv
- 태양광 발전 예측 기상.csv
- 태양광자원정보.csv
- 통합자원정보.csv


In [ ]:
# 3. 로드할 CSV 자동 선택 (우선순위: 전력사용량 관련)

# 1) 파일명에 특정 키워드가 들어간 CSV를 우선 선택한다.
    # 혹시 파일명이 조금 달라도 자동으로 잡히게 키워드 후보를 여러 개 둔다.
keywords = ["전력", "사용", "시간", "법정동", "load", "power"]

# 2) data. 폴더에서 CSV만 추린다.
csv_files = sorted(DATA_DIR.glob("*.csv"))

if len(csv_files) == 0:
    raise FileNotFoundError(f"data 폴더({DATA_DIR}) 안에 CSV 파일 없으니 확장자가 .csv 인지 확인")

# 3) 키워드 매칭 점수로 "가장 그럴듯한" 파일을 고른다.
def score_filename(name: str, keywords: list[str]) -> int:
    return sum(1 for k in keywords if k in name)

scored = [(f, score_filename(f.name, keywords)) for f in csv_files]
scored_sorted = sorted(scored, key = lambda x: x[1], reverse = True)

LOAD_FILE = scored_sorted[0][0] # 점수가 가장 높은 파일
print("오늘 로드할 파일 =", LOAD_FILE.name)
print("경로 =", LOAD_FILE)

오늘 로드할 파일 = 법정동별시간별전력사용량.csv
경로 = c:\Users\user\Desktop\seoul_pv_load_analysis\data\법정동별시간별전력사용량.csv


In [ ]:
# 4. CSV 로드 (인코딩 이슈 대비)

# 1) 한국어 CSV는 인코딩 때문에 에러 날 수 있다.
    # 그래서 여러 인코딩을 순서대로 시도한다.
encodings_to_try = ["utf-8-sig", "cp949", "euc-kr"]

df_raw = None
last_error = None

for enc in encodings_to_try:
    try:
        df_raw = pd.read_csv(LOAD_FILE, encoding = enc)
        print(f"CSV 로드 성공! (encoding = {enc})")
        break
    except Exception as e:
        last_error = e

# 2) 끝까지 실패하며 에러를 보여주고 멈춘다.
if df_raw is None:
    raise RuntimeError("CSV 로드 실패. 인코딩 / 구분자 문제일 수 있음"
                       f"마지막 에러: {last_error}")

# 3) 데이터가 어떻게 생겼는지 "맛보기" 확인
print("행/열 크기 =", df_raw.shape)
display(df_raw.head(5))

CSV 로드 성공! (encoding = utf-8-sig)
행/열 크기 = (9754804, 5)


,SIGUNGU_CD,BJDONG_CD,USE_YM,USE_HM,FDRCT_VLD_KWH
0,11650,10700,20220628,100,10782.0565
1,11650,10800,20220628,100,11394.8635
2,11650,10900,20220628,100,7273.9620
3,11740,10300,20220628,100,11008.8110
4,11710,11300,20220628,100,2905.1120


In [9]:
# 5. 컬럼 구조 점검

# 1) 컬럼 이름을 전부 본다.
print("컬럼 목록:")
for c in df_raw.columns:
    print("-", c)

# 2) 결측치(빈 값) 비율을 간단히 본다. (상위 20개)
na_ratio = (df_raw.isna().mean() * 100).sort_values(ascending = False)
print("\n 결측치 비율 TOP 20 (%):")
display(na_ratio.head(20))

컬럼 목록:
- SIGUNGU_CD
- BJDONG_CD
- USE_YM
- USE_HM
- FDRCT_VLD_KWH

 결측치 비율 TOP 20 (%):


SIGUNGU_CD       0.0
BJDONG_CD        0.0
USE_YM           0.0
USE_HM           0.0
FDRCT_VLD_KWH    0.0
dtype: float64

In [ ]:
# 6. 시간 컬럼 찾기 + datetime 만들기

'''
목표: 
- 데이터에서 "시간 정보가 담긴 컬럼"을 찾아서
- datetime 으로 변환하고
- hour(0 ~ 23), month(1 ~ 12)를 뽑을 수 있는지 확인한다.
'''

# 1) 시간 컬럼 후보 키워드
time_like_keywords = ["일시", "일자", "날짜", "date", "time", "datetime", "측정", "기준"]

# 2) 컬럼명에 키워드가 들어간 후보를 찾는다.
time_candidates = [c for c in df_raw.columns if any(k.lower() in str(c).lower() for k in time_like_keywords)]
print("시간 컬럼 후보:", time_candidates)

# 3) 후보가 없으면: 
    # - 날짜 / 시간이 별도 컬럼이 아니라
    # - 이미 "시간대(0 ~ 23)" 컬럼만 있을 수도 있다.
    # 그래서 hour 후보도 따로 찾아본다.
hour_keywords = ["시", "hour", "시간"]
hour_candidates = [c for c in df_raw.columns if any(k.lower() in str(c).lower() for k in hour_keywords)]
print("시간대(hour) 후보:", hour_candidates)

시간 컬럼 후보: []
시간대(hour) 후보: []


In [ ]:
# 7. 우선 1개 시가 컬럼을 골라  datetime 변환 시도
    # 데이터마다 컬럼명이 다를 수 있어서, 자동으로 가장 유력한 후보 1개 잡기

# 1) 시간이 들어있을 가능성이 가장 높은 컬럼을 하나 고른다.
    # - time_candidates가 있으면 그 중 첫 번쨰를 사용
    # - 없으면 datetime 변환은 일단 건너뛰고 hour 만으로 MVP 진행
dt_col = time_candidates[0] if len(time_candidates) > 0 else None
print("선택된 시간 컬럼=", dt_col)

df = df_raw.copy()

if dt_col is not None:
    # 2) datetime 변환 시도
    # errors = "coerce" 는 변환 실패 한 값은 NaT(결측)로 만들고 넘어가게 해준다.
    df["datetime"] = pd.to_datetime(df[dt_col], errors = "coerce" )

    # 3) 변화 성공률 체크
    success_rate = df["datetime"].notna().mean() * 100
    print(f"datetime 변환 성공률 = {success_rate:.1f}%")

    # 4) 성공률이 너무 낮으면 컬럼 선택이 잘못됐을 확률이 높다.
    # - 이 경우엔 컬렁명을 보고 직접 지정하는 게 더 빠르다.
    if success_rate < 50:
        print("datetime 변환 성공률이 낮으므로 시간 컬럼이 아닐 수 있다.")
        print(" -> time_candidates 목록 중 다른 컬럼을 시도해야 할 수 있다.")

    # 5) 시간 파생 변수 만들기
    df["hour"] = df["datetime"].dt.hour
    df["month"] = df["datetime"].dt.month

else:
    print("시간 컬럼을 못 찾음. DAY 1은 hour 컬럼 기반으로 진행해야 함.")

선택된 시간 컬럼= None
시간 컬럼을 못 찾음. DAY 1은 hour 컬럼 기반으로 진행해야 함.


In [15]:
# 8. DAY 1 최소 분석 테이블 (df_base) 만들기
# 목적: "시간축만 제대로 만들면 DAY 1 성공", 이 단계에선 아직 "완벽한 전처리" 안 함.

# 1) 분석에 꼭 필요한 컬럼만 모은다.
    # - datetime / hour / month + 전력사용량(숫자) 컬럼이 무엇인지 찾아야 한다.

# 2) 전력 사용량 후보 키워드로 컬럼 찾기
value_keywords = ["사용", "전력", "kwh", "kw", "부하", "load", "power", "사용량"]
value_candidates = [c for c in df.columns if any(k.lower() in str(c).lower() for k in value_keywords)]
print("전력 / 사용량 후보 컬럼:", value_candidates)

# 3) 후보가 여러 개면, 일단 첫 번째로 가정하고 진행
    # DAY 1 목적은 "가능성 확인" 이니까 우선 진행해보고 DAY 2  확정
value_col = value_candidates[0] if len(value_candidates) > 0 else None
print("선택된 값 컬럼 = ", value_col)

if value_col is None:
    print("전력 사용량 컬럼을 자동으로 찾지 못함")
    print(" -> 컬럼 목록에서 '전력 사용량'에 해당하는 컬렁명을 직접 지정해야 함")
else:
    # 4) 값 컬럼을 숫자로 변환한다.
        # - 문자 / 쉼표가 섞여 있을 수 있어서 to_numeric 을 사용
    df["value"] = pd.to_numeric(df[value_col].astype(str).str.replace(",", ""), errors = "coerce")
 
    # 5) df_base 구성
    base_cols = []
    if "datetime" in df.columns: base_cols.append("datetime")
    if "month" in df.columns: base_cols.append("month")
    if "hour" in df.columns: base_cols.append("hour")
    base_cols.append("value")

    df_base = df[base_cols].copy()

    # 6) 기초 점검
    print("df_base 크기 =", df_base.shape)
    print("value 결측 비율(%) =", df_base["value"].isna().mean() * 100)

    display(df_base.head(10))

전력 / 사용량 후보 컬럼: ['FDRCT_VLD_KWH']
선택된 값 컬럼 =  FDRCT_VLD_KWH
df_base 크기 = (9754804, 1)
value 결측 비율(%) = 0.0


,value
0,10782.0565
1,11394.8635
2,7273.9620
3,11008.8110
4,2905.1120
5,3416.9310
6,10473.6130
7,14196.5290
8,5894.3225
9,6393.6870


In [16]:
# 9. DAY 1 종료 체크

# DAY 1 성공 기준: 
    # - df_base가 존재한다.
    # - hour가 0 ~ 23 범위를 가진다. (또는 대부분이 그 범위에 들어간다.)
    # - value가 숫자로 들어왔다.

if "df_base" in globals():
    print("DAY 1 체크 1) df_base 생성: OK")

    if "hour" in df_base.columns:
        h_min, h_max = df_base["hour"].min(), df_base["hour"].max()
        print(f"DAY 1 체크 2) hour 범위: {h_min} ~ {h_max}")
    else:
        print("DAY 1 체크 2) hour 컬럼 없음 (시간 컬럼 / 파싱 문제 가능)")
    
    v_non_na = df_base["value"].notna().mean() * 100
    print(f"DAY 1 체크 3) value 숫자 성공률: {v_non_na:.1f}%")

    print("\n DAY 1 끝!")

else:
    print('df_base가 없어서 DAY 1 실패')
    print(" -> time / value 컬럼 선택을 다시 해야 한다.")

DAY 1 체크 1) df_base 생성: OK
DAY 1 체크 2) hour 컬럼 없음 (시간 컬럼 / 파싱 문제 가능)
DAY 1 체크 3) value 숫자 성공률: 100.0%

 DAY 1 끝!
